In [ ]:
#from azureml.core import Dataset
import os
import shutil
import tempfile
from auth import get_auth
from azureml.core import Workspace, Experiment, Run
#from azureml.core.dataset import Dataset
import glob

In [ ]:
code_dir = "src"
paths = glob.glob(os.path.join(code_dir, "*.py"))
paths

In [ ]:
#create a temp folder and copy code, model and dataset

In [ ]:
print("Creating temp folder...")
temp_path = "tmp_eval"
if os.path.exists(temp_path):
    shutil.rmtree(temp_path)
os.mkdir(temp_path)

for p in paths:
    shutil.copy(p, temp_path)
print("Done.")

In [ ]:
MODEL_EXPERIMENT_NAME = 'q3-depthmap-plaincnn-height-95k'
#MODEL_RUN_ID = 'q3-depthmap-plaincnn-height-95k_1597988908_42c4ef33'
MODEL_RUN_ID = 'q3-depthmap-plaincnn-height-95k_1600451633_cb44f6db'
MODEL_INPUT_LOCATION = 'outputs/best_model.h5'
MODEL_OUTPUT_LOCATION = temp_path
EVAL_DATASET_NAME = 'anon-depthmap-testset'
DATASET_TARGET_PATH = temp_path + 'data'

In [ ]:
def download_model(ws, EXPERIMENT_NAME, RUN_ID, INPUT_LOCATION, OUTPUT_LOCATION):
    experiment = Experiment(workspace = ws, name = EXPERIMENT_NAME)
    #Download the model on which evaluation need to be done
    run = Run(experiment, run_id = RUN_ID)
    #run.get_details()
    run.download_file(INPUT_LOCATION, OUTPUT_LOCATION)
    print("Successfully downloaded model")

In [ ]:
ws = Workspace.from_config(auth = get_auth())

In [ ]:
download_model(ws, MODEL_EXPERIMENT_NAME, MODEL_RUN_ID, MODEL_INPUT_LOCATION, MODEL_OUTPUT_LOCATION)

In [ ]:
EVAL_EXPERIMENT_NAME = "QA-pipeline"
tags = {}
experiment = Experiment(workspace = ws, name = EVAL_EXPERIMENT_NAME)

In [ ]:
#Find/create a compute target.

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "gpu-cluster"

# Compute cluster exists. Just connect to it.
try:
    compute_target = ComputeTarget(workspace = ws, name=cluster_name)
    print("Found existing compute target.")

# Compute cluster does not exist. Create one.    
except ComputeTargetException:
    print("Creating a new compute target...")
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='Standard_NC6', 
        max_nodes=4
    )
    compute_target = ComputeTarget.create(workspace, cluster_name, compute_config)
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)
    
compute_target

In [ ]:
dataset = ws.datasets[EVAL_DATASET_NAME]
dataset

In [ ]:
from azureml.core.runconfig import MpiConfiguration
from azureml.train.dnn import TensorFlow
TensorFlow.get_supported_versions()

In [ ]:
from src.config import CONFIG

script_params = {f"--{k}": v for k, v in CONFIG.items()}
script_params

In [ ]:
# Specify pip packages here.
pip_packages = [
    "azureml-dataprep[fuse,pandas]",
    "glob2",
    "opencv-python==4.1.2.30",
    "matplotlib",
]

# Create the estimator.
estimator = TensorFlow(
    source_directory=temp_path,
    compute_target=compute_target,
    entry_script="evaluate.py",
    use_gpu=True,
    framework_version="2.2",
    inputs=[dataset.as_named_input("dataset").as_mount()],
    pip_packages=pip_packages,
    script_params=script_params
)

# Set compute target.
estimator.run_config.target = compute_target

# Run the experiment.
run = experiment.submit(estimator, tags=tags)

# Show run.
run

In [ ]:
run.wait_for_completion(show_output=True)

In [ ]:
print(run.get_status())

In [ ]:
CSV_SAVE_LOCATION = '.'
run.download_file(CONFIG.CSV_OUT_PATH, CSV_SAVE_LOCATION)
print("Downloaded the result2.csv")

In [ ]:
#Delete temp folder
shutil.rmtree(temp_path)